# MA3J8 Approximation Theory and Applications

## 01 - Introductory Example

In this notebook we will use numerical quadrature to explore some elementary examples demonstrating interesting aspects of approximation theory that we will explore throughout Part I of the module.

All numerical examples will be shown in [Julia](https://julialang.org). They can be viewed but not executed using [nbviewer](http://nbviewer.jupyter.org). Alternatively, students can install Julia 1.0 (download from link above), then install [IJulia.jl](https://github.com/JuliaLang/IJulia.jl)

This notebook requires 2 external packages, which can be installed from the Julia package manager. Open a Julia REPL, press `]` to see the package manager prompt `(v1.0) pkg>` and then 
```
add Plots 
add QuadGK
```

In [ ]:
using Plots, QuadGK

### 01.1 - Quadrature Error: Algebraic vs Exponential

We will numerically approximate the integrals 
$$
   I_j := \int_{-\pi}^\pi f_j(x) \,dx,
$$
where $f_j : [-\pi, \pi] \to \mathbb{R}$ are given by 
$$\begin{aligned}
   f_1(x) &:= \frac{1}{1 + x^2}, \\ 
   f_2(x) &:= \frac{1}{1 + \sin^2(x)}
\end{aligned}$$

Both of these functions belong to $C^\infty(-1, 1)$ with "least smoothness" near the origin, where they agree to second order.

In [ ]:
# define the two functions f1, f2 and plot them on a fine grid
#  NOTE: if X is a collection and f a functions then f.(X) == [f(x) for x in X]
#        this is called "broadcasting" in Julia; see the documentation for more detail     
f1(x) = 1 / (1 + x^2)
f2(x) = 1 / (1 + sin(x)^2) 
a, b = -π, π
xx = range(a; stop=b, length=200)
plot(xx, f1.(xx), lw=2, label = "f1")
plot!(xx, f2.(xx), lw=2, label = "f2")

We use the trapezoidal rule to approximate the integrals:  Let $N > 0$, $h = 2\pi/N$ and $x_n = -\pi + n h$ then 
$$\begin{aligned}
  I_j &= \sum_{n = 1}^N \int_{x_{n-1}}^{x_n} f_j(x) \, dx \\ 
  &\approx \sum_{n = 1}^N \frac{h}{2} \big( f_j(x_{n-1}) + f_j(x_n) \big) =: Q_{j,N}
\end{aligned}$$

In [ ]:
# implementation of the trapezoidal rule 
function trapezoidal_rule(f, a, b, N)
    x = range(a; stop=b, length=N+1)   # linspace(a, b, N+1)
    h = (b-a)/N
    return h * sum( f.(x) ) - h/2 * (f(a)+f(b)) 
end ;

we can compute approximate integrals and what how they converge

In [ ]:
@show trapezoidal_rule(f1, a, b, 10)
@show trapezoidal_rule(f1, a, b, 20)
@show trapezoidal_rule(f1, a, b, 30)
@show trapezoidal_rule(f1, a, b, 40)
@show trapezoidal_rule(f1, a, b, 50)
;

In [ ]:
@show trapezoidal_rule(f2, a, b, 10)
@show trapezoidal_rule(f2, a, b, 20)
@show trapezoidal_rule(f2, a, b, 30)
;

This indicates that the trapezoidal rule converges in both cases, but *much* faster for $f_2$ than for $f_1$. We can make this much more visible by plotting the errors against the number of grid points.

In [ ]:
# numer of grid points for f1
NN1 = (2).^(2:10)
# high accuracy numerical solution for f1
I_f1, _ = quadgk(f1, a, b, rtol = 1e-12) 
# quadrature errors of the trapezoidal rule
err_f1 = [ abs(trapezoidal_rule(f1, a, b, N) - I_f1)  for N in NN1 ]

# numer of grid points for f1
NN2 = 2:2:20
# high accuracy numerical solution for f1
I_f2, _ = quadgk(f2, a, b, rtol = 1e-12) 
# quadrature errors of the trapezoidal rule
err_f2 = [ abs(trapezoidal_rule(f2, a, b, N) - I_f2)  for N in NN2 ]

# plot the convergence both on a log-log and and a lin-log axis
P1 = plot(NN1, err_f1, lw=2, m=:o, label = "|I1 - Q1N|")
plot!(P1, NN2, err_f2, lw=2, m=:o, label = "|I2 - Q2N|")
plot!(P1; xaxis = (:log,), yaxis = (:log,))
P2 = plot(NN1, err_f1, lw=2, m=:o, label = "|I1 - Q1N|")
plot!(P2, NN2, err_f2, lw=2, m=:o, label = "|I2 - Q2N|")
plot!(P2; xaxis = ([0, 2*NN2[end]], ), yaxis = (:log,))
plot(P1, P2)

When one first encounters these results, they should be very puzzling; what is going on? Qualitatively the two functions $f_1, f_2$ seem very similar, in particular both are $C^\infty$. The only difference seems to be that $f_2$ is periodic while $f_1$ is not? (In fact, $f_1$ is also periodic, but it is no longer smooth when interpreted as a periodic function)

We need to dive a bit into the analysis of the trapezoidal rule to explain our results. The motivation of the trapezoidal rule is the following figure.

![Illustration of the Trapzoidal Rule](figs/trapezoidal_rule.pdf)

On some interval $[x_{j-1}, x_j]$, the integral under the curve is approximated by the integral under the straight line, which can be computed analytically. On a larger interval, the interal is split into subintervals $[x_{j-1}, x_j]$ and the trapzoidal rule is applied to each subinterval.

To understand the error in the trapezoidal rule, let $x_0 = x_{j-1}, x_1 = x_j, h = x_1-x_0$ and define 
$$
  \tilde{f} := f(x_0) \frac{x_1 - x}{h} + f(x_1) \frac{x - x_0}{h}
$$
then $\tilde{f}$ is the affine interpolant in the figure. We can now evaluate the interpolation error. Let $g := f - \tilde{f}$ then $g(x_0) = 0$ hence there exists $\xi \in (x_0, x_1)$ such that $g(x) = g'(\xi) (x - x_0$.
Since $\int_{x_0}^{x_1} g'(x) = 0$ there exists $\eta$ such that 
$g'(\eta) = 0$ and in particular, 
$$
  g'(\xi) = g'(\xi) - g'(\eta) = g''(\zeta) (\xi - \eta).
$$
In summary, since $g'' = f''$, we have 
$$
    |f(x) - \tilde{f}(x)| \leq h^2 \|f''\|_\infty.
$$
This interpolation error readily implies [EXERCISE] that 
$$
    |I_j - Q_{jN}| \lesssim N^{-2}.
$$
Indeed we can test this numerically.

In [ ]:
using LaTeXStrings

In [ ]:
plot(NN1, err_f1, lw=2, m=:o, label = L"|I[f_1] - Q_N[f_1]|")
plot!(NN2, err_f2, lw=2, m=:o, label = L"|I[f_2] - Q_N[f_2]|")
plot!(NN1[3:end-2], NN1[3:end-2].^(-2), lw=2, ls=:dash, c=:black, label = "N^-2")
plot!(; xaxis = (:log,), yaxis = (:log,))

The question remains how we can explain the *much faster* convergence rate of $Q_{2N}$. 
To that end we need to recall the Fourier series, formally for now, 
$$
   f(x) = \sum_{k \in \mathbb{Z}} {\widehat{f}}_k e^{i k x}
$$
Then, since $\int_{-\pi}^\pi e^{-ikx} dx = 0$ for all $k \in \mathbb{Z} \setminus \{0\}$, we have 
$$
  Q_{2,N} =2 \pi \widehat{f}_0
$$

**Observation 1:** [Proof is an exercise!]  
$$
    \sum_{n = 1}^N e^{i k x_n} = 0 \quad \text{for all $k \not\in N \mathbb{Z}$}
$$
and $= N$ otherwise.

Therefore, using periodicity, we obtain 
$$\begin{aligned}
   Q_{2N} &= h \sum_{n = 1}^N f(x_n) \\ 
   &= h \sum_{j = 1}^N \sum_{k \in \mathbb{Z}} \widehat{f}_k e^{ik x_j} \\ 
   &= h \sum_{k \in \mathbb{Z}} \widehat{f}_k \sum_{j=1}^N e^{ik x_j} \\ 
   &= 2 \pi \widehat{f}_0 + h \sum_{k \in N \mathbb{Z} \setminus \{0\}} \widehat{f}_k N.
\end{aligned}$$
Thus, the quadrature error is 
$$
  |I_2 - Q_{2,N}| \leq 2 \pi \sum_{k \in N \mathbb{Z} \setminus \{0\}} \big|\widehat{f}_k\big|.
$$

This is an entirely different kind of error estimate that does not involve the local regularity of the function $f_2$ but the decay of its Fourier coefficients. We are not quite equipped yet to analyse this decay, but the following observation will give some intuition: 
$$
   f^{(m)}(x) = \sum_{k \in \mathbb{Z}} (i k)^m \widehat{f}_k e^{i k x},
$$
and if $f^{(m)}$ is "regular" (e.g., integrable) then this implies that the 
coefficients $(i k)^m \widehat{f}_k$ must be bounded, and in particular, 
$$
   \big|\widehat{f}_k\big| \lesssim |k|^{-m}.
$$
Indee we will later prove a much stronger result, the Paley-Wiener theorem, that for analytic functions, 
$$
    \big|\widehat{f}_k\big| \lesssim e^{-\alpha |k|}.
$$
Suppose we have proven this, then it is now straightforward to establish that 
$$
    |I_2 - Q_{2,N}| \lesssim e^{-\alpha N},
$$
which is precisely what we observed in our numerical results.

**EXERCISE:** look up the fourier transforms of various functions of different degree of smoothness (e.g., google for `table of fourier transforms`) and observe how the regularity of the function affects the fourier coefficients, e.g., 
$$\begin{aligned}
    \hat\delta(k) &= 1, \\ 
    \widehat{{\rm sign}}(k) &= \frac{2}{ik}, \\ 
    f(x) = e^{-|x|} \Rightarrow \hat{f}(k) &= \frac{2}{1+k^2}, \\ 
    f(x) = e^{- \pi x^2} \Rightarrow \hat{f}(k) &= e^{-\pi k^2} 
\end{aligned}$$